## качаем данные

In [1]:
import requests
from bs4 import BeautifulSoup 
import time
from urllib.parse import urlparse, urljoin
from tqdm.notebook import tqdm

In [2]:
## найти id каждого вопроса на страницах
page = 1
main_url = 'https://zpp.rospotrebnadzor.ru/Forum/Appeals/AjaxindexList?page={page}&searchtext=&categories=[]'

def ids_from_page(soup):
    return [i.get('href') for i  in soup.findAll('a', {'class':'appeal-title-link'})]
    
urls = []
for page in tqdm(range(1, 50), desc='Pages'):
    time.sleep(0.5)
    res = requests.get(main_url.format(page=page))  
    soup = BeautifulSoup(res.text)
    urls.extend(ids_from_page(soup))

Pages:   0%|          | 0/49 [00:00<?, ?it/s]

In [3]:
## имея id запросов достанем тексты на страницах
domain = 'https://zpp.rospotrebnadzor.ru'

def get_text_page_question(soup):
    return soup.findAll('p', {'class':'appeal-details-message'})[0].text
def get_text_page_answer(soup):
    return soup.findAll('p', {'class':'appeal-comments-message'})[0].text

question = []
answers = []

for url in tqdm(urls, desc='Pages'):
    time.sleep(0.5)
    res = requests.get(urljoin(domain, url))  
    soup = BeautifulSoup(res.text)
    
    question.append(get_text_page_question(soup))
    try:
        answers.append(get_text_page_answer(soup))
    except:
        answers.append('No answer')

Pages:   0%|          | 0/245 [00:00<?, ?it/s]

In [6]:
## посмотрим, что скачалось
N = 2
print('Q:', question[N])
print()
print('A:', answers[N])

Q: Магазин Sunlight отказывается возвращать средства за электронный подарочный сертификат, ссылаясь на правила пользования сертификатами установленные самим магазином. На прикрепленных скриншотах суть вопроса показана ещё подробнее, как и ответ самого магазина.

A: Добрый день! Подарочную карту или сертификат можно приобрести практически в любом магазине.  При этом номинал подарочной карты определяется потребителем. Важно помнить, что подарочная карта не является товаром, а лишь подтверждает внесение денежных средств за товар, который может быть приобретен как самим плательщиком, так и другим лицом в последующем в пределах суммы, доступной на подарочной карте.
 
Сущностная ценность и предназначение подарочных карт заключается в предоставлении возможности одним лицом другому лицу выбрать товар из ассортимента магазина, выпустившего подарочную карту, на сумму, зачисленную на подарочную карту, соответственно, подарочные карты являются особым объектом гражданского оборота.
 
Когда приобрет